In [88]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from IPython.display import display, HTML

In [89]:
game = pd.read_csv("business-data/game_data.csv")
player = pd.read_csv("business-data/player_data.csv")
training = pd.read_csv("business-data/training_set.csv")
test = pd.read_csv("business-data/test_set.csv")

In [90]:
game.head()

,Season,Game_ID,Game_Date,Team,Location,Wins_Entering_Gm,Losses_Entering_Gm,Team_Minutes,Final_Score,Lead_Changes,Ties,Largest_Lead,Full_Timeouts,Short_Timeouts,Qtr_4_Score,L2M_Score
0,2016-17,21600001,10/25/2016,CLE,H,0,0,240.0,117.0,1.0,3.0,32.0,4.0,1.0,82.0,113.0
1,2016-17,21600001,10/25/2016,NYK,A,0,0,240.0,88.0,0.0,1.0,4.0,4.0,2.0,64.0,84.0
2,2016-17,21600002,10/25/2016,POR,H,0,0,240.0,113.0,6.0,6.0,13.0,5.0,0.0,77.0,104.0
3,2016-17,21600002,10/25/2016,UTA,A,0,0,240.0,104.0,5.0,1.0,8.0,5.0,2.0,83.0,102.0
4,2016-17,21600003,10/25/2016,GSW,H,0,0,240.0,100.0,1.0,2.0,4.0,3.0,1.0,77.0,100.0


In [91]:
player.head()

,Season,Game_ID,Game_Date,Team,Person_ID,Name,ASG_Team,Active_Status,Minutes,Points,...,Steals,Blocks,Turnovers,Field_Goals,Field_Goals_Attempted,Three_Pointers,Three_Pointers_Attempted,Free_Throws,Free_Throws_Attempted,Personal_Fouls
0,2016-17,21600001,10/25/2016,CLE,101112,Channing Frye,None,Active,14.0,6.0,...,1.0,1.0,1.0,2.0,6.0,2.0,5.0,0.0,0.0,4.0
1,2016-17,21600001,10/25/2016,CLE,201567,Kevin Love,East ASG,Active,25.0,23.0,...,3.0,0.0,2.0,6.0,15.0,2.0,6.0,9.0,12.0,3.0
2,2016-17,21600001,10/25/2016,CLE,202681,Kyrie Irving,East ASG,Active,30.0,29.0,...,2.0,0.0,0.0,12.0,22.0,4.0,7.0,1.0,1.0,1.0
3,2016-17,21600001,10/25/2016,CLE,202684,Tristan Thompson,None,Active,22.0,0.0,...,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0
4,2016-17,21600001,10/25/2016,CLE,202697,Iman Shumpert,None,Active,14.0,2.0,...,0.0,1.0,2.0,1.0,3.0,0.0,1.0,0.0,0.0,2.0


In [92]:
training.count()

Season             280625
Game_ID            280625
Game_Date          280625
Away_Team          280625
Home_Team          280625
Country            280625
Rounded Viewers    280625
dtype: int64

In [93]:
viewership = training.groupby(["Season", "Game_ID", "Game_Date", "Away_Team", "Home_Team"]).sum()
viewership.reset_index(inplace=True)
print(viewership.count())
viewership.head()

Season             2000
Game_ID            2000
Game_Date          2000
Away_Team          2000
Home_Team          2000
Rounded Viewers    2000
dtype: int64


,Season,Game_ID,Game_Date,Away_Team,Home_Team,Rounded Viewers
0,2016-17,21600001,10/25/2016,NYK,CLE,57691
1,2016-17,21600002,10/25/2016,UTA,POR,40740
2,2016-17,21600003,10/25/2016,SAS,GSW,60428
3,2016-17,21600004,10/26/2016,MIA,ORL,22756
4,2016-17,21600005,10/26/2016,DAL,IND,33922


In [94]:
test.head()

,Season,Game_ID,Game_Date,Away_Team,Home_Team,Total_Viewers
0,2016-17,21600010,10/26/2016,DEN,NOP,NaN
1,2016-17,21600023,10/28/2016,HOU,DAL,NaN
2,2016-17,21600027,10/29/2016,BOS,CHA,NaN
3,2016-17,21600026,10/29/2016,ATL,PHI,NaN
4,2016-17,21600048,11/1/2016,LAL,IND,NaN


In [95]:
test.count()

Season           460
Game_ID          460
Game_Date        460
Away_Team        460
Home_Team        460
Total_Viewers      0
dtype: int64

# DataFrame Pre-Processing and Matrix Extraction

In [96]:
game.head()

,Season,Game_ID,Game_Date,Team,Location,Wins_Entering_Gm,Losses_Entering_Gm,Team_Minutes,Final_Score,Lead_Changes,Ties,Largest_Lead,Full_Timeouts,Short_Timeouts,Qtr_4_Score,L2M_Score
0,2016-17,21600001,10/25/2016,CLE,H,0,0,240.0,117.0,1.0,3.0,32.0,4.0,1.0,82.0,113.0
1,2016-17,21600001,10/25/2016,NYK,A,0,0,240.0,88.0,0.0,1.0,4.0,4.0,2.0,64.0,84.0
2,2016-17,21600002,10/25/2016,POR,H,0,0,240.0,113.0,6.0,6.0,13.0,5.0,0.0,77.0,104.0
3,2016-17,21600002,10/25/2016,UTA,A,0,0,240.0,104.0,5.0,1.0,8.0,5.0,2.0,83.0,102.0
4,2016-17,21600003,10/25/2016,GSW,H,0,0,240.0,100.0,1.0,2.0,4.0,3.0,1.0,77.0,100.0


In [97]:
len(np.unique(game["Game_ID"].tolist()))

2460

In [98]:
def game_processing(data):
    """
    Extracts only relevant column for analysis from game data.
    Returns new dataframe.
    """
    
    data = game.copy(deep=True)
    teams = ["LAL","GSW","MIA","CHI","SAS","BOS","CLE","OKC","HOU","NYK","ORL","TOR","DAL","PHI","LAC", "ATL", "POR", "IND", "PHO", "SAC", "MEM", "BKN", "CHA", "MIL", "MIN", "WAS", "DET", "NOP", "UTA", "DEN"]

    for team in teams:
        data[team] = data["Team"] == team
        data[team] = data[team]*1
    #Calculated Win_Percentage
    data["Win_Percentage"] = np.divide(data["Wins_Entering_Gm"], data["Wins_Entering_Gm"] + data["Losses_Entering_Gm"])
    data.fillna(value=0, inplace=True)

    #Extract Holidays
    data["Game_Date"] = pd.to_datetime(data["Game_Date"], infer_datetime_format=True)
    cal = calendar()
    holidays = cal.holidays()
    data['Holiday'] = data['Game_Date'].isin(holidays)
    data["Holiday"] = data["Holiday"] * 1

    data["Monday"] = data["Game_Date"].dt.weekday == 0
    data["Monday"] = data["Monday"] * 1

    data["Tuesday"] = data["Game_Date"].dt.weekday == 1
    data["Tuesday"] = data["Tuesday"] * 1

    data["Wednesday"] = data["Game_Date"].dt.weekday == 2
    data["Wednesday"] = data["Wednesday"] * 1

    data["Thursday"] = data["Game_Date"].dt.weekday == 3
    data["Thursday"] = data["Thursday"] * 1

    data["Friday"] = data["Game_Date"].dt.weekday == 4
    data["Friday"] = data["Friday"] * 1

    data["Saturday"] = data["Game_Date"].dt.weekday == 5
    data["Saturday"] = data["Saturday"] * 1

    data["Sunday"] = data["Game_Date"].dt.weekday == 6
    data["Sunday"] = data["Sunday"] * 1

    #Extract Top-10 Media Markets // source: http://www.nielsen.com/content/dam/corporate/us/en/public%20factsheets/tv/2017-18%20TV%20DMA%20Ranks.pdf
    top_markets = ["NYK", "LAL", "BKN", "LAC", "CHI", "TOR", "PHI", "DAL", "WAS", "HOU", "GSW", "ATL", "BOS"]
    data["Top Markets"] = data["Team"].isin(top_markets)
    data["Top Markets"] = data["Top Markets"]*1

    #Combine win_percentages from 2 teams(2 rows) into 1 row (record)
    max_win = data.groupby("Game_ID")["Win_Percentage"].max()
    min_win = data.groupby("Game_ID")["Win_Percentage"].min()

    data.drop(["Location", "Win_Percentage", "Team", "Game_Date", "Wins_Entering_Gm", "Losses_Entering_Gm", "Season", "Team_Minutes", "Final_Score", "Lead_Changes", "Ties", "Largest_Lead", "Full_Timeouts", "Short_Timeouts", "Qtr_4_Score", "L2M_Score"], axis=1, inplace=True)

    data = pd.DataFrame(data.groupby('Game_ID').agg({'Holiday':'first', 'Monday': 'first', 'Tuesday': 'first', 'Wednesday': 'first', 'Thursday': 'first', 'Friday': 'first', 'Saturday': 'first', 'Sunday': 'first', 'Top Markets':'sum', "LAL": 'max', "GSW": 'max',"MIA": 'max',"CHI": 'max',"SAS": 'max',"BOS": 'max',"CLE": 'max',"OKC": 'max',"HOU": 'max',"NYK": 'sum', "ORL": 'sum',"TOR": 'sum',"DAL": 'sum',"PHI": 'sum',"LAC": 'sum', "ATL": 'sum', "POR": 'sum', "IND": 'sum', "PHO": 'sum', "SAC": 'sum', "MEM": 'sum', "BKN": 'sum', "CHA": 'sum', "MIL": 'sum', "MIN": 'sum', "WAS": 'sum', "DET": 'sum', "NOP": 'sum', "UTA": 'sum', "DEN": 'sum'}))
    data["Higher Win Percentage"] = max_win.tolist()
    data["Lower Win Percentage"] = min_win.tolist()
    data.reset_index(inplace=True)

    data.set_index("Game_ID", inplace=True)


    return data

game_data = game_processing(game)
game_data

/Users/rohansuresh/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


,Holiday,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Top Markets,LAL,...,CHA,MIL,MIN,WAS,DET,NOP,UTA,DEN,Higher Win Percentage,Lower Win Percentage
Game_ID,,,,,,,,,,,,,,,,,,,,,
21600001,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0.000000,0.000000
21600002,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0.000000,0.000000
21600003,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0.000000,0.000000
21600004,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000000,0.000000
21600005,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0.000000,0.000000
21600006,0,0,0,1,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0.000000,0.000000
21600007,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0.000000,0.000000
21600008,0,0,0,1,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0.000000,0.000000
21600009,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0.000000,0.000000


In [99]:
def player_processing(playerdata):
    def helper(a, b):
        if a == "East ASG" or a == "West ASG":
            if b == "Active":
                return 1
        else:
            return 0
    playerdata["Active All Stars"] = playerdata.apply(lambda row: helper(row["ASG_Team"], row["Active_Status"]), axis=1)
    group = playerdata.groupby(["Game_ID"])["Active All Stars"].sum()
    playerdata = pd.DataFrame(group)
    return playerdata

player_data = player_processing(player)
player_data.head()


,Active All Stars
Game_ID,
21600001,4.0
21600002,0.0
21600003,5.0
21600004,0.0
21600005,1.0


In [100]:
#You should have 1/2 number of rows in game and player dataframes now
assert 2*len(np.unique(game_data.index.tolist())) == len(game["Game_ID"].tolist())

In [101]:
def split_inputs_outputs(set_type, training=True):
    """
    Completes the data by taking in a dataframe 'data' as input, and outputs all game- and player-related
    data related to each game in 'data'. For training.
    
    Input: data (Training or Test Set)
    Output: Complete data (X input), Viewership (Y output)
    """
    
    def mergedata(df1, df2):
        """
        Takes 2 data dataframes, and outputs one giant matrix of input values (X) to be trained for parameters by merging(horizontally)
        the dataframes. Join on Index.
        """
        return df1.join(df2, how="inner")
    
    training_set = set_type.copy(deep=True)
    training_set.set_index("Game_ID", inplace=True)

    X = mergedata(mergedata(game_data, player_data), training_set)
    if training==False:
        X.fillna(value=0, inplace=True)
        y = X["Total_Viewers"].tolist()
        X.drop(["Season", "Game_Date", "Away_Team", "Home_Team", "Total_Viewers"], axis=1,inplace=True)
    else:
        y = X["Rounded Viewers"].tolist()
        X.drop(["Season", "Game_Date", "Away_Team", "Home_Team", "Rounded Viewers"], axis=1,inplace=True)
    
    
    
    return X, y

X, y = split_inputs_outputs(viewership)


In [102]:
assert len(y) == len(X)

# Training Data (for Convenience)

In [103]:
X.head()

,Holiday,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Top Markets,LAL,...,MIL,MIN,WAS,DET,NOP,UTA,DEN,Higher Win Percentage,Lower Win Percentage,Active All Stars
Game_ID,,,,,,,,,,,,,,,,,,,,,
21600001,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0.0,0.0,4.0
21600002,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0.0,0.0,0.0
21600003,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0.0,0.0,5.0
21600004,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0.0
21600005,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0.0,0.0,1.0


# Combine Training Data and Test

In [104]:
test_X, test_y =  split_inputs_outputs(test, training=False)

In [105]:
full_X = pd.concat([X, test_X])
full_y = y + test_y

# Lasso & Ridge Regression


In [106]:
#Convert DataFrame to Matrix
X = X.as_matrix()
print(X.shape)

(2000, 42)


In [107]:
full_X = full_X.as_matrix()
print(full_X.shape)

(2460, 42)


In [108]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1)
clf.fit(X, y)
lasso = list(clf.predict(X))
lasso

[43156.21465190637,
 11911.466462669661,
 44832.35120028416,
 8718.636470377349,
 9601.11389928358,
 18850.20785653333,
 13045.459710233219,
 12878.78320209619,
 10749.763046711152,
 26838.935459343622,
 8241.519121769828,
 25693.700797477894,
 17084.515403328296,
 26069.082584265343,
 15463.018357056837,
 17629.369583948897,
 35216.93111129006,
 10112.948368966396,
 8929.965284141215,
 19930.783426459253,
 5463.820383776854,
 16727.438680991345,
 36783.9798850455,
 16828.107488167258,
 34933.16076240986,
 13731.892916491768,
 13664.319550696704,
 19043.997830926623,
 9023.975958613857,
 13147.253289491302,
 12397.275634884287,
 34985.547242017514,
 16776.73619642683,
 10949.115839558346,
 14064.78505663851,
 28622.028002631956,
 18634.201378775695,
 12119.545120406874,
 10364.88065557648,
 7636.097428258556,
 11943.51484299554,
 16112.922124006896,
 40297.51430881584,
 14660.448042676278,
 11903.487494685642,
 33448.15817512323,
 15407.789560994732,
 13919.35926065026,
 4504.740531478

In [109]:
def total_MAPE(pred):
    s = 0
    for actual, predicted in zip(y, pred):
        s += np.abs(actual - predicted)/actual
    return (1/len(y))*s
print("Lasso Error: " + str(total_MAPE(lasso)))

Lasso Error: 0.3524364530344074


In [110]:
from sklearn.linear_model import Ridge
clf = Ridge(alpha=1.0)
clf.fit(X, y) 
ridge = list(clf.predict(X))
print("Ridge Error: " + str(total_MAPE(ridge)))

Ridge Error: 0.3532175954606984


Due to the slightly smaller error, we will proceed in making our test set predictions with lasso regression.

In [111]:
test_X.shape


(460, 42)

In [112]:
clf_lasso = linear_model.Lasso(alpha=0.1)
clf_lasso.fit(X, y)
lasso_predictions = list(clf_lasso.predict(test_X))
lasso_predictions


[9590.062393676511,
 18293.75496985094,
 17421.736732706864,
 18901.48935186943,
 19130.03668827121,
 10130.192050187965,
 12745.51350049039,
 15483.998010350982,
 18844.321971684985,
 7852.989747365677,
 14670.915215636862,
 45134.979835396836,
 18950.910643061274,
 7219.2808033955525,
 16908.177158500446,
 10377.159747201498,
 21850.433553809984,
 3995.9431190443593,
 34796.788522126095,
 38315.193303270324,
 18140.992426807024,
 17277.516003179437,
 13414.975252613965,
 24421.448909666968,
 8099.905141729032,
 33189.19816806655,
 17726.32184270492,
 19822.077868413355,
 22528.45756369479,
 32402.378888209114,
 11079.892823655475,
 9898.679452702514,
 14959.579767745745,
 5999.047948322207,
 19780.28249814491,
 16689.060920136348,
 15587.610490906729,
 7026.9122380481695,
 14116.85306387934,
 15173.87098195789,
 16340.199799266707,
 10766.595794435194,
 18446.401897824715,
 19818.485084245884,
 8576.2533380575,
 10500.154605212803,
 13194.64072423992,
 13949.789110683845,
 10724.2304

In [113]:

clf_ridge = Ridge(alpha=1.0)
clf_ridge.fit(X, y) 
ridge_predictions = list(clf_ridge.predict(test_X))
ridge_predictions

[9377.987200203326,
 18111.51512015091,
 17450.63685229601,
 18903.646009493565,
 19056.206367344887,
 10152.66181773576,
 12780.698453175752,
 15390.250268277181,
 18977.33827946962,
 8034.050560496115,
 14680.686657216613,
 45079.53530311673,
 19049.213480562885,
 7194.987406405533,
 16813.651119053513,
 10510.580023799277,
 21850.15154281252,
 4277.3832548699675,
 34745.54046871135,
 38149.123605837056,
 17957.094475871258,
 17302.096371810097,
 13357.790526525318,
 24346.46676855121,
 8018.545078328369,
 33201.25949922248,
 17581.612798319944,
 19632.744010283997,
 22369.203830338654,
 32355.084055120013,
 11075.961168760517,
 10077.480144405436,
 14845.500478678454,
 6210.945601310651,
 19692.518113405036,
 16900.489703780884,
 15668.525956178793,
 7160.478350753053,
 14032.353205064137,
 15136.534381737682,
 16293.026658954128,
 10850.26335399404,
 18467.408820218716,
 19647.385076784427,
 8851.184187732262,
 10440.192989970392,
 13270.291008399854,
 13989.28123451625,
 10743.904

In [118]:
# Writing lasso_predictions to the test data
test["Total_Viewers"] = lasso_predictions
test.head()

,Season,Game_ID,Game_Date,Away_Team,Home_Team,Total_Viewers
0,2016-17,21600010,10/26/2016,DEN,NOP,9590.062394
1,2016-17,21600023,10/28/2016,HOU,DAL,18293.754970
2,2016-17,21600027,10/29/2016,BOS,CHA,17421.736733
3,2016-17,21600026,10/29/2016,ATL,PHI,18901.489352
4,2016-17,21600048,11/1/2016,LAL,IND,19130.036688


In [119]:
# Write test dataframe back to csv
test.to_csv('./business-data/test_set_BigBallerPredictions.csv')